<a href="https://colab.research.google.com/github/byrongomezjr/AAPLStockForecastingWithLSTMs/blob/main/AAPLStockForecastingWithLSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#load dataset

In [5]:
#load dataset
from google.colab import files
dataset = files.upload()

Saving aapl.csv to aapl.csv


In [16]:
df = pd.read_csv('aapl.csv')

In [17]:
df.shape

(6081, 7)

In [18]:
df = df['Open'].values
df = df.reshape(-1,1)

In [32]:
df.shape

(6081, 1)

In [21]:
dataset_train = np.array(df[:int(df.shape[0]*0.8)])
dataset_test = np.array(df[int(df.shape[0]*0.8):])

print(dataset_train.shape)
print(dataset_test.shape)

(4864, 1)
(1217, 1)


#importing the module

In [33]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout

#scaling data

In [35]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset_train = scaler.fit_transform(dataset_train)
dataset_test = scaler.transform(dataset_test)
dataset_train[:5]

array([[0.55074317],
       [0.4883597 ],
       [0.38793884],
       [0.4286932 ],
       [0.38905759]])

In [37]:
dataset_test = scaler.transform(dataset_test)
dataset_test[:5]

array([[-0.06781841],
       [-0.06778294],
       [-0.06774037],
       [-0.06773327],
       [-0.06775456]])

# creating our datasets

In [36]:
def create_dataset(df):
  x = []
  y = []
  for i in range(50, df.shape[0]):
    x.append(df[i-50:i, 0])
    y.append(df[i, 0])
  x = np.array(x)
  y = np.array(y)
  return x,y

# training and testing data

In [38]:
x_train, y_train = create_dataset(dataset_train)
x_test, y_test = create_dataset(dataset_test)

# reshaping our data, turning it into a 3D array

In [39]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

#model building

In [40]:
model = Sequential()
model.add(LSTM(units=96, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=96, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=96, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=96))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [41]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [42]:
model.compile(loss='mean_squared_error', optimizer='adam')